# HW 6: Sentiment Analysis

In this question, you'll need dataset:
- `hw7_train.csv`: dataset fro training
- `hw7_test.csv`: dataset for test

A snippet of the dataset is given below.



In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import logging
import string
import nltk
from sklearn import metrics
from gensim.models import word2vec
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.svm import LinearSVC

In [2]:

train = pd.read_csv("hw6_train.csv")
test = pd.read_csv("hw6_test.csv")

train.head(10)
#test.head(10)

,label,text
0,1,Getting ready for college. I had a good sleep....
1,1,We are having a party now to have all the fami...
2,0,@marC0110 ummm.. i see you.. and i really wann...
3,1,@saboteur1 Thanks for following Much apprecia...
4,1,Why eat at home? Picnic plans for today are al...
5,0,I hear a puppy crying
6,0,@lollipop26 Hi! Is there any chance you know o...
7,0,@Grumpydev Probably stretch to a new iPhone ca...
8,0,"@AmberMVaughan I know, that's totally the down..."
9,0,Just got the furthest I've ever got on Sonic 2...


## Q1: Unsupervised Sentiment Analysis

Write a function `analyze_sentiment(docs, labels, th)` as follows:
- Takes three inputs:
   - `docs` : a list of documents, 
   - `labels` the ground-truth sentiment labels of `docs`
   - `th`: compound threshold
- Use Vader to get a compound score of for each document in `docs`.  
- If `compound score > th`, then the predicted label is 1; otherwise 0
- Print out the classification report
- This function has no returns

In [3]:
def analyze_sentiment(docs, labels, th=0):
    pred_label = []
    sia = SentimentIntensityAnalyzer()
    docs = docs.tolist()
    for i in range(len(docs)):
        sentiment_score = sia.polarity_scores(docs[i])
        if sentiment_score['compound'] > th:
            pred_label.append(1)
        else: 
            pred_label.append(0)
    #print(pred_label[:10])
    
    print(metrics.classification_report(labels, pred_label))
    return None
    

In [4]:
analyze_sentiment(test["text"], test["label"], 0)

              precision    recall  f1-score   support

           0       0.64      0.68      0.66      9968
           1       0.66      0.62      0.64     10032

    accuracy                           0.65     20000
   macro avg       0.65      0.65      0.65     20000
weighted avg       0.65      0.65      0.65     20000



## Q2: Supervised Sentiment Analysis Using Word Vectors

### Q2.1: Train Word Vectors

Write a function `train_wordvec(docs, vector_size)` as follows:
- Take two inputs:
    - `docs`: a list of documents
    - `vector_size`: the dimension of word vectors
- First tokenize `docs` into tokens
- Use `gensim` package to train word vectors. Set the `vector size` and also carefully set other parameters such as `window`, `min_count` etc.
- return the trained word vector model

In [5]:
def train_wordvec(docs, vector_size = 100):
    
    wv_model = None
    sentences=[ [token.strip(string.punctuation).strip() \
             for token in nltk.word_tokenize(doc.lower()) \
                 if token not in string.punctuation and \
                 len(token.strip(string.punctuation).strip())>=2]\
             for doc in docs]
    
    
    # print out tracking information
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                    level=logging.INFO)
    
    wv_model = word2vec.Word2Vec(sentences, \
            min_count=5, vector_size = vector_size, \
            window=5, workers=4 )
    
    return wv_model

In [6]:
wv_model = train_wordvec(train["text"], vector_size = 100)
print(wv_model)

2022-12-18 22:58:06,275 : INFO : collecting all words and their counts
2022-12-18 22:58:06,275 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-12-18 22:58:06,289 : INFO : PROGRESS: at sentence #10000, processed 124321 words, keeping 18776 word types
2022-12-18 22:58:06,304 : INFO : PROGRESS: at sentence #20000, processed 248579 words, keeping 31136 word types
2022-12-18 22:58:06,319 : INFO : PROGRESS: at sentence #30000, processed 373660 words, keeping 41982 word types
2022-12-18 22:58:06,334 : INFO : PROGRESS: at sentence #40000, processed 497842 words, keeping 51722 word types
2022-12-18 22:58:06,350 : INFO : PROGRESS: at sentence #50000, processed 622677 words, keeping 61076 word types
2022-12-18 22:58:06,366 : INFO : collected 69759 word types from a corpus of 746531 raw words and 60000 sentences
2022-12-18 22:58:06,367 : INFO : Creating a fresh vocabulary
2022-12-18 22:58:06,388 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retain

Word2Vec<vocab=7830, vector_size=100, alpha=0.025>


### Q2.2: Generate Vector Representation for Documents

Write a function `generate_doc_vector(docs, wv_model)` as follows:
- Take two inputs:
    - `docs`: a list of documents, 
    - `wv_model`: trained word vector model. Set the default value to 100.
- First tokenize each document `doc` in `docs` into tokens
- For each token in `doc`, look up for its word vector in `wv_model`. Then the document vector (denoted as `d`) of `doc` can be calculated as the `mean of the word vectors of its tokens`, i.e. $d = \frac{\sum_{i \in doc}{v_i}}{|doc|}$, where $v_i$ is the word vector of the i-th token.
- Return the vector representations of all `docs` as a numpy array of shape `(n, vector_size)`, where `n` is the number of documents in `docs` and `vector_size` is the dimension of word vectors.


Note: It may not be a good idea to represent a document as the mean of its word vectors. For example, if one word is positive and another is negative, the sum of the these two words may make the resulting vector is no longer sensitive to sentiment. You'll learn more advanced methods to generate document vector in deep learning courses.

In [7]:
def generate_doc_vector(docs, wv_model):
    
    vectors = []
    
    sentences=[[token.strip(string.punctuation).strip() \
             for token in nltk.word_tokenize(doc.lower()) \
                 if token not in string.punctuation and \
                 len(token.strip(string.punctuation).strip())>=2]\
             for doc in docs]
    
    for sent in sentences:
        token_vector = []
        for word in sent:
            if word in wv_model.wv.key_to_index:
                token_vector.append(wv_model.wv[word])
        vectors.append(token_vector)
        
    d = np.ndarray((len(vectors), 100))
    
    for i in range(len(vectors)):
        d[i] = np.mean(vectors[i], axis = 0)
        
    print(d.shape)
    

    return d

In [8]:
train_X = generate_doc_vector(train["text"], wv_model)
test_X = generate_doc_vector(test["text"], wv_model)

/Users/subhuatharva/miniconda3/envs/my-env/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/subhuatharva/miniconda3/envs/my-env/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(60000, 100)
(20000, 100)


In [9]:
t = pd.DataFrame(train_X)

In [10]:
t = t.fillna(0)

In [11]:
t.shape

(60000, 100)

In [12]:
train_X = generate_doc_vector(train["text"], wv_model)
test_X = generate_doc_vector(test["text"], wv_model)

(60000, 100)
(20000, 100)


### Q2.3: Put everything together


Define a function `predict_sentiment(train_text, train_label, test_text, test_label, vector_size = 100)` as follows:

- Take the following inputs:
    - `train_text, train_label`: a list of documents and their labels for training
    - `test_text, test_label`: a list of documents and their labels for testing,
    - `vector_size`: the dimension of word vectors. Set the default value to 100.
- Call `train_wordvec(docs, vector_size)` to train a word vector model using `train_text`
- Call `generate_doc_vector(docs, wv_model)` to generate vector representations (denoted as `train_X`) for documents in `train_text`. 
- Call `generate_doc_vector(docs, wv_model)` to generate vector representations (denoted as `test_X`) for each document in `test_text`
- Fit a linear SVM model using `train_X` and `train_label`
- Predict the label for `test_X` and print out classification report for the testing subset.
- This function has no return

### Q2.4: Analysis

- Compare the classification reports you obtain from Q1 and Q2.3. Which model performs better?
- Why this model can achieve better performance?

In [13]:
def predict_sentiment(train_text, train_label, test_text, test_label, vector_size = 100):
    
    wv_model = train_wordvec(train_text, vector_size)
    
    train_X = generate_doc_vector(train_text, wv_model)
    train_X = pd.DataFrame(train_X)
    train_X = train_X.fillna(0)
    
    test_X = generate_doc_vector(test_text, wv_model)
    test_X = pd.DataFrame(test_X)
    test_X = test_X.fillna(0)
    
    classifier = LinearSVC()
    classifier.fit(train_X, train_label)
    
    y_pred = classifier.predict(test_X)
    
    print(metrics.classification_report(test_label, y_pred))
    
    return None

In [14]:
predict_sentiment(train["text"], train["label"],\
                  test["text"], test["label"],\
                  vector_size = 100)
    

2022-12-18 22:58:25,864 : INFO : collecting all words and their counts
2022-12-18 22:58:25,864 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-12-18 22:58:25,878 : INFO : PROGRESS: at sentence #10000, processed 124321 words, keeping 18776 word types
2022-12-18 22:58:25,892 : INFO : PROGRESS: at sentence #20000, processed 248579 words, keeping 31136 word types
2022-12-18 22:58:25,907 : INFO : PROGRESS: at sentence #30000, processed 373660 words, keeping 41982 word types
2022-12-18 22:58:25,921 : INFO : PROGRESS: at sentence #40000, processed 497842 words, keeping 51722 word types
2022-12-18 22:58:25,937 : INFO : PROGRESS: at sentence #50000, processed 622677 words, keeping 61076 word types
2022-12-18 22:58:25,953 : INFO : collected 69759 word types from a corpus of 746531 raw words and 60000 sentences
2022-12-18 22:58:25,954 : INFO : Creating a fresh vocabulary
2022-12-18 22:58:25,975 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retain

(60000, 100)
(20000, 100)
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      9968
           1       0.71      0.73      0.72     10032

    accuracy                           0.71     20000
   macro avg       0.71      0.71      0.71     20000
weighted avg       0.71      0.71      0.71     20000



/Users/subhuatharva/miniconda3/envs/my-env/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [15]:
if __name__ == "__main__":  
    
    train = pd.read_csv("hw6_train.csv")
    test = pd.read_csv("hw6_test.csv")
    
    # Test Q1
    
    print("Test Q1")
   
    analyze_sentiment(test["text"], test["label"], 0)
    
    print("\n==================\n")
    print("Test Q2")
    
    predict_sentiment(train["text"], train["label"],\
                  test["text"], test["label"],\
                  vector_size = 100)

Test Q1
              precision    recall  f1-score   support

           0       0.64      0.68      0.66      9968
           1       0.66      0.62      0.64     10032

    accuracy                           0.65     20000
   macro avg       0.65      0.65      0.65     20000
weighted avg       0.65      0.65      0.65     20000



Test Q2


2022-12-18 22:59:04,423 : INFO : collecting all words and their counts
2022-12-18 22:59:04,424 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-12-18 22:59:04,437 : INFO : PROGRESS: at sentence #10000, processed 124321 words, keeping 18776 word types
2022-12-18 22:59:04,452 : INFO : PROGRESS: at sentence #20000, processed 248579 words, keeping 31136 word types
2022-12-18 22:59:04,467 : INFO : PROGRESS: at sentence #30000, processed 373660 words, keeping 41982 word types
2022-12-18 22:59:04,482 : INFO : PROGRESS: at sentence #40000, processed 497842 words, keeping 51722 word types
2022-12-18 22:59:04,497 : INFO : PROGRESS: at sentence #50000, processed 622677 words, keeping 61076 word types
2022-12-18 22:59:04,512 : INFO : collected 69759 word types from a corpus of 746531 raw words and 60000 sentences
2022-12-18 22:59:04,512 : INFO : Creating a fresh vocabulary
2022-12-18 22:59:04,535 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retain

(60000, 100)
(20000, 100)
              precision    recall  f1-score   support

           0       0.72      0.71      0.71      9968
           1       0.71      0.73      0.72     10032

    accuracy                           0.72     20000
   macro avg       0.72      0.72      0.72     20000
weighted avg       0.72      0.72      0.72     20000



/Users/subhuatharva/miniconda3/envs/my-env/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
